# 도전 키워드 추출!
- 무엇을 도전하나요?
    - 각 리뷰 토큰화
    - 각 상품별 리뷰를 총집합하여 최적의 키워드 n개 추출
- 무슨 방식으로 하나요?
    - CountVectorizer
    - TF-IDF
    - (라이브러리)KoNLPy
    - (라이브러리)KRWordRank

### 1. 각종 라이브러리 import!

In [1]:
# 파일시스템 관련
import os

# 데이터 처리 관련
import numpy as np
import pandas as pd

# 텍스트 전처리 관련
import re
import string
from konlpy.tag import Kkma
# stopwords도 알아서 구해볼것

# 데이터시각화 관련
import matplotlib.pyplot as plt
%matplotlib inline

# sklearn
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

__불용어 정리__

In [2]:
ko_stopwords_raw = """아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다"""

In [3]:
ko_stopwords = list(ko_stopwords_raw.split('\n'))
ko_stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

### 시작

In [4]:
df_raw = pd.read_excel("2차통합(리뷰수조정포함)_전체_리뷰데이터.xlsx")
df_raw.shape

(793223, 14)

In [12]:
df = df_raw

In [13]:
# df = df.drop_duplicates() # 중복행 제거
# df = df.dropna()          # 결측치 제거
df.shape

(793223, 14)

In [14]:
df.columns.tolist()

['Unnamed: 0',
 '상품명',
 '가격',
 '총점',
 '리뷰수',
 '날짜',
 '아이디',
 '별점',
 '내구성',
 '가격.1',
 '디자인',
 '배송',
 '좋아요',
 '내용']

In [15]:
# 상품명, 아이디, 내용 제외한 컬럼 제거
df = df[['상품명', '내용']] # 선택한 column만 들고온다
print(df.shape)
df.head(2)

(793223, 2)


,상품명,내용
0,크리스마스 LED 테이퍼 초 블랙폿 테이블 센터피스 장식 소품,오브제 덕분에 지인들과 짧지만 좋은 시간을 가졌습니다:) 디테일도 마음에 들고 확실...
1,크리스마스 LED 테이퍼 초 블랙폿 테이블 센터피스 장식 소품,브라운을 시켰는데 레드가 왔구요..보니 빨간 열매 한알이 떨어져있네요 예쁘긴한데 내...


In [16]:
# column명 변경
df.columns = ['pname', 'review']

# rating 컬럼 숫자로 대체
# df['rating'] = df['rating'].apply(lambda x: x.split('점')[1].lstrip())

df.head(1)

,pname,review
0,크리스마스 LED 테이퍼 초 블랙폿 테이블 센터피스 장식 소품,오브제 덕분에 지인들과 짧지만 좋은 시간을 가졌습니다:) 디테일도 마음에 들고 확실...


### 리뷰 텍스트 전처리


In [17]:
# 구두점 특수문자 제거
def text_preprocess(text):
    text = re.sub('\[.*!?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # 구두점 제거
    text = re.sub('\n', '', text)
    return text

In [19]:
df['review'] = df['review'].apply(lambda x: text_preprocess(str(x)))
df.head(3)

<ipython-input-19-7ac77e141a2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: text_preprocess(str(x)))


,pname,review
0,크리스마스 LED 테이퍼 초 블랙폿 테이블 센터피스 장식 소품,오브제 덕분에 지인들과 짧지만 좋은 시간을 가졌습니다 디테일도 마음에 들고 확실히 ...
1,크리스마스 LED 테이퍼 초 블랙폿 테이블 센터피스 장식 소품,브라운을 시켰는데 레드가 왔구요보니 빨간 열매 한알이 떨어져있네요 예쁘긴한데 내구성...
2,도일리 드림캐쳐(2size),저는 속커튼사이에 두려고 구매했어요 생각했던 그대로여서 너무만족합니다 좋은꿈 꾸겠죠


# konlpy 자연어처리 라이브러리

### Kkma, Okt 로 형태소 분해

In [60]:
kkma = Kkma()
kkma.morphs(u"""배송이 좀 늦네요 ㅠㅠ 하지만 택배포장은 굿입니당""")

['배송', '이', '좀', '늦', '네요', 'ㅠㅠ', '하지만', '택배', '포장', '은', '굿', '이', 'ㅂ니다']

In [61]:
from konlpy.tag import Okt
okt = Okt()
okt.morphs(u"""배송이 좀 늦네요 ㅠㅠ 하지만 택배포장은 굿입니당""")

['배송', '이', '좀', '늦네요', 'ㅠㅠ', '하지만', '택배', '포장', '은', '굿', '입니당']

__성능 => Okt가 더 낫다!__

In [62]:
# df['pname'].value_counts().index
df['pname'].value_counts()

문체어 3colors                            222
네이쳐 원목스툴 12colors                      216
클린업 터치 휴지통 + 분리수거 비닐봉투 20L 20매입        208
WH 화이트 원형테이블 800size                   206
줄무늬 글라스 유리병 2size                      204
                                      ... 
차단이 잘되는 비욘드 화이트 나비주름 쉬폰 먼지없는 항균 속커튼      1
다옴 1인 식기세트 6p                            1
크리스마스트리 중대형 풀세트 트리 모음                    1
심플 모던 깔끔한 화이트 린넨 가리개 커튼                  1
앵두전구 크리스마스 벽트리 장식 세트 100구/11m            1
Name: pname, Length: 360, dtype: int64

### reviews -> 한 상품에 대한 전체 리뷰를 리스트화

In [64]:
reviews = [r for r in df_todo['review']]
len(reviews)

222

In [65]:
reviews[:5]

['배송하루만에 왔네요 펼치면 되는 완제품이구 짐옮기거나 이사할때 짐덩이 안되구 좋아요 양반다리 할정도 크기는 아니지만 베란다에 두려고 일부러 작은사이즈 했는데 불편하지 않아요 체구 있는 신랑도 보기와 다르게 편하대요',
 '편하게 쉴 수 의자이고 접이식이라서 공간 활용에 좋습니다',
 '디자인 가격 사이즈 전부 마음에 들지만 뚠뚠이 신랑이 앉기에 약한 내구성과 두개를 구매했는데 한개만 배송 후 홈페이지에 두개 모두 배송 완료라고 표기해놓은 것 때문에 별 두개 뺐습니다 의자는 넘넘 마음에 들어요',
 '딸 방에 놓으려고 샀는데 울 양이가 더 좋아하는거 같네요 골골거리고 안내려오네요',
 '그리너리한 느낌 만땅으로 너무 잘어울립니다 감사합니다']

# CountVectorizer or TextRank + TF-IDF 사용

### unigram

In [ ]:
# 캐글발 코드
vec = CountVectorizer(stop_words=ko_stopwords).fit(reviews)
bag_of_words = vec.transform(reviews)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x:x[1], reverse=True)

In [69]:
words_freq[:10]

[('좋아요', 55),
 ('너무', 44),
 ('생각보다', 43),
 ('편하고', 38),
 ('편해요', 28),
 ('정말', 18),
 ('ㅎㅎ', 17),
 ('마음에', 14),
 ('좋네요', 14),
 ('샀는데', 13)]

### bigram

In [70]:
vec2 = CountVectorizer(ngram_range=(2,2), stop_words=ko_stopwords).fit(reviews)
bag_of_words2 = vec2.transform(reviews)
sum_words2 = bag_of_words2.sum(axis=0)
words_freq2 = [(word, sum_words2[0, idx]) for word, idx in vec2.vocabulary_.items()]
words_freq2 = sorted(words_freq2, key = lambda x:x[1], reverse=True)
words_freq2[:10] # 상위 10개

[('편하고 좋아요', 10),
 ('마음에 들어요', 7),
 ('좋아요 좋아요', 6),
 ('너무 편하고', 6),
 ('쓰고 있어요', 6),
 ('너무 편해서', 5),
 ('생각보다 너무', 5),
 ('둥근 디자인의', 4),
 ('좋아요 짱짱짱좋아요', 4),
 ('커서 좋았어요', 4)]

## Okt 형태소 분석기로 각 리뷰 토큰화한다음 uni, bigram 만들기

In [75]:
text = reviews[0]
text

'배송하루만에 왔네요 펼치면 되는 완제품이구 짐옮기거나 이사할때 짐덩이 안되구 좋아요 양반다리 할정도 크기는 아니지만 베란다에 두려고 일부러 작은사이즈 했는데 불편하지 않아요 체구 있는 신랑도 보기와 다르게 편하대요'

In [76]:
text = okt.morphs(text)

In [80]:
print(text)

['배송', '하루', '만에', '왔네요', '펼치면', '되는', '완제', '품이구', '짐', '옮기거나', '이사', '할', '때', '짐', '덩이', '안되구', '좋아요', '양반', '다리', '할', '정도', '크기', '는', '아니지만', '베란다', '에', '두', '려고', '일부러', '작은', '사이즈', '했는데', '불편하지', '않아요', '체구', '있는', '신랑', '도', '보기', '와', '다르게', '편하', '대요']


In [79]:
print(reviews[0].split())

['배송하루만에', '왔네요', '펼치면', '되는', '완제품이구', '짐옮기거나', '이사할때', '짐덩이', '안되구', '좋아요', '양반다리', '할정도', '크기는', '아니지만', '베란다에', '두려고', '일부러', '작은사이즈', '했는데', '불편하지', '않아요', '체구', '있는', '신랑도', '보기와', '다르게', '편하대요']


In [84]:
print(okt.nouns(reviews[0])) # 명사만 빼기

['배송', '하루', '완제', '짐', '이사', '때', '짐', '덩이', '양반', '다리', '정도', '크기', '베란다', '려고', '일부러', '사이즈', '체구', '신랑', '보기', '대요']


## 9만여개 리뷰 상품별로 정리하여 리스트로 만들기

In [153]:
df = df_raw
df.shape

(93976, 14)

In [154]:
df = df.drop_duplicates()
df.shape

(92995, 14)

In [155]:
df = df.dropna()
df.shape

(92993, 14)

In [ ]:
# df 3컬럼만 빼내기
print(df.columns.tolist())
df = df[['상품명', '아이디', '내용']]
df.shape

In [157]:
df.columns = ['pname', 'uid', 'review']
df.head(1)

,pname,uid,review
0,디퓨저 리필액 200ml 1+1+1+1 + 캔들증정,톨레란쓰,딱..제가 좋아하는 빨래 향 ㅜㅜ 작은 방에 뒀더니 상쾌한 냄새나요.. 빨래 널어서...


In [158]:
# 상품명 별로 정렬하기
df = df.sort_values(by='pname')

# 상품개수 몇개인지 알아보기
# df['pname'].nunique() # unique value count
df['pname'].value_counts()

메리미 이중레이스 암막커튼 3size 3colors                   1000
원목 사이드테이블 협탁 (강화유리선반)                          1000
Romane 발매트 shape type 6종 택1                    1000
애슐리 전신거울 3colors                               1000
포송 양귀비 가지 조화_7color                            1000
                                               ... 
올리브 조화 나무 화분세트                                    1
디퓨저 섬유스틱 화이트                                      1
행운 4K UHD LED TV 65인치 HW55KUGEL (스탠드 방문설치)        1
극강의 편안함 무중력 우주방석 / 강아지방석 (프리미엄_겨울용) 3colors       1
달빛식기 전용 유리볼 L (단품)                                1
Name: pname, Length: 412, dtype: int64

In [186]:
# text_preprocess()함수로 리뷰내용에서 구두점 등등 제외하기
df['review'] = df['review'].apply(lambda x: text_preprocess(x))
df.head(10)

,pname,uid,review
14463,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),서원범,암막도 엄청좋고 배송도 빠르고 색도 너무 이뻐요
14345,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),mive1y,커튼봉이 늦게와서 커튼을 늦게달았는데 저는 너무이쁘고 맘에들거든요근데엄마가 무슨장례...
14344,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),ㅇㅇ1238648,생각한 그대로이고 주름만 스팀으로 펴면 될거같아요
14343,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),pungel,깔끔하고 전체적으로 집이 넓어 보이네요 완전 화이트 원했는데 딱이에요
14342,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),듀듀ㄷ두,색도 딱이고 두께도 좋아요 저는 암막 목적보다 우풍 막으려고 산거라 살짝 빛은 비치...
14341,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),유댕덩덩,화면과 같은 색상이며 빛 차단도 잘 되는것 같아 만족합니다
14340,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),찡여사_,딸아이방에 달아주려 구매하였습니다 넘 핑크핑크하지않고 인디핑크색이라 더 맘에 들어요...
14339,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),김총명5,좀짧습니다 227 보시면되고 저희집은 층고가 240 은되야해서 좀 짧네요 참고하세요
14338,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),냐하아ㅏㅏㅏ,디자인이나 색감 가격 너무 맘에 드는데 반듯한 네모 커튼이 아니라 울퉁불퉁하고 마감...
14337,(1+1) 솔리드 방한 암막커튼 핀형/아일렛형 (작은창/긴창/대형),gustn9045,그냥 그럭저럭 쓸만해요 길이가 쫌만 길었음 좋았을거 같아요


In [25]:
# 412 개의 상품별 row구조로 변환
df_rev = pd.DataFrame(df['pname'].value_counts())
df_rev.reset_index(level=0, inplace=True) # pname이 index로 가서 index를 column으로 변환시켜줌
df_rev.columns = ['pname', 'count']
df_rev.head(10)

,pname,count
0,선데이 러그 7size 4colors,33619
1,순수원목 선반장 4colors (가로2단),29225
2,오로라 단스탠드(전구 증정 이벤트),20335
3,애슐리 전신거울 3colors,14545
4,"집순이 슬리브 옥스포드 삼각 등쿠션 (솜포함, 2size / 6color)",14491
5,소프트 모던 러그/카페트 4colors,14169
6,순수 원목 선반 신발장 2단,11407
7,헤링본 사이잘룩 라탄 러그 카페트 4colors 4size,9523
8,BRUG 스탠드 행거 3colors,8957
9,가벽인테리어 스트라이프 파티션 4size 5colors(리뷰이벤트),6457


In [26]:
df_rev.shape

(3650, 2)

In [27]:
print(df_rev[:2])

                     pname  count
0     선데이 러그 7size 4colors  33619
1  순수원목 선반장 4colors (가로2단)  29225


In [28]:
first_prod = '메리미 이중레이스 암막커튼 3size 3colors'

# pname이 first_prod인 리뷰들을 리스트에 모으기
df_first = df[df['pname']==first_prod]
df_first['review'].tolist()

['햇빛 암막 100프로 되구요이쁜건 덤이에여 진짜 강추완전강추 이 가격에 이런디자인에 이런 암막기능이면 얼른사야해요집분위기도 완전 호텔방처럼 너무이뻐요👍🏻👍🏻제발 사세요그리고 세로길이는 233cm라고해서 길겟다햇는데 저희집 227cm 밑에가아니고 윗부분이 길어서 다행히 끌리지않고 딱맞는데 잘 보고 구매하셔야할거같아요',
 '커튼을 잘근 잘근 밟아 왔네요 도저히 드러워서 빨기도 싫고 확인좀 하고 보내지 어떻게 관리하길래 슬리퍼 자국 선명하고 더럽더럽 저런때는 빨아도안지워짐 반품 교환 구찬은데빨리 다시 보내주세요 짜증짜증',
 '우리집 고영희가 좋아해요 그럼 다 된거에요 이만한 가성비 오지는 제품이 없습니다 판매자님 많이 팔아 부자되시고 건강하십시오',
 '만족해요 샤라라 하네요 근데 천 질이 그렇게 좋지는 않아요',
 '환해보이고 싶으면 화이트 추천 입니다',
 '이걸 도대체 어떻게 설명 해야하는지후배송 받자마자 너무 싼마이가 확어떻게 묶어봐도 싸보여서그 값 한다 생각하다가도 보면 화날정도ㅠㅠ그래서 커튼집 가서 새로 삿지뭐에요 속 천은 폴리천이어서 구김도 바스락바스락 겉 망사는 이걸 어떻게 표현해야되지부드러운 망사가 아닌아너무별로임오늘의집 어플 없애고싶을 정도에요ㅠㅠ 진짜 긍정왕인데ㅠㅠㅠㅠㅠ빨리 새로 산 커튼 왔으면 좋겠어요여러분 사진에 속지말자구여 방금 새로 찍은건데도 그 느낌이 안잡힙니당',
 '커튼 진짜 이뻐요 진심 웨딩드레스처럼 촤르르 떨어져요',
 '색도 예쁘고 러블리하고 좋아용 설치도 쉽고 고리형이여서 가려야하나 걱정했는데 커튼이예뻐서 고리도 별로 가릴필요없네요 작은창 용으로도 있으면 구매했을텐데 아쉬워요 ㅠㅠ',
 '저렴한가격과 이쁜디자인과 빠른배송 너무 마음에드네요 번창하세요',
 '화이트톤의 암막커튼이라 고민많이했는데 이중으로 레이스도 있어서 인테리어와 사생활보호및 햇빛차단까지 완벽하니 전혀 후회없고 만족합니다 ㅎㅎ',
 '아이가 무조건이뿐거 고집해서 좀많이망설였어요 아이가좋아하니 나두 좋네요',
 '우선 제가 설치한곳은 베란다쪽이구요 가로길이

In [32]:
# 각 상품별 리뷰만 모아 리턴하는 함수
def gain_reviews_per_product(pname):
    return df[df['pname']==pname]['review'].tolist()

In [193]:
print(gain_reviews_per_product('애슐리 전신거울 3colors')[:10])

['아빠가 ㅇ오만원은 줬겠다하셨어요 대만족', '클래식하고 분위기 있네요 재구매 의사 있습니다', '조립도 쉽고 너무 예뻐요 저렴한 가격에 배송도 괜찮았습니다 잘쓰겠습니다 많이 파세용', '이젠 너무 흔해진 거울이지만 그래도 엔틱한느낌이 너무 예쁘고 좋습니다 다만 남자들이 보기에는 거울이 조금 짧다고 그러네요', '친구에게 선물한건데 무척 만족스러워한다조립도 간단하고 여러모로 쓸모가 잇다', '너무예뻐용 근데 이거 받아보시면 알겠지만 끼워져있던 봉이 너무 꽉조여졌어서 온가족이 스패너들고 거울과 싸웠답미당호호 거울 자체는 예뻐욥', '고급스럽고 이뻐서 아주 맘에 듭니다 잘샀어요ㅎ', '거울 너무 이쁘네요 월넛색 빈티지하면서 절대 이가격으러 안보여요 ㅎㅎㅎ 다행히 스크래치 이런것도 없는거로 잘 왔네요 조립은 두명이서하는게 수월할듯', '예뻐요 근데 지지대가 조금 약한느낌 쨋든 전체적으로 예뻐요', '너무 좋아요 급하게 필요해서 그냥 베스트상품 샀는데 괜히 베스트가 아니네요 조립도 완전 쉬워요']


__정상적으로 상품별 리뷰가 추출된다__

### df_rev에 각 상품별 리뷰 list 붙이기

In [30]:
# 상품명 리스트 따로 빼기
pname_list = df_rev.pname.tolist()
print(len(pname_list), pname_list[:3])

3650 ['선데이 러그 7size 4colors', '순수원목 선반장 4colors (가로2단)', '오로라 단스탠드(전구 증정 이벤트)']


## 각 상품별로 모든 리뷰 리스트에 모으는 쉬운 코드
`d.groupby('A')['B'].apply(list)`

In [33]:
# 상품별로 리뷰 모으기
reviews = []
for pname in pname_list:
    reviews.append(gain_reviews_per_product(pname))

In [34]:
df_rev['review'] = reviews
df_rev.shape

(3650, 3)

In [36]:
df_rev.head()

,pname,count,review
0,선데이 러그 7size 4colors,33619,"[너무 좋네요 선배가 와서 양념치킨 먹다가 바로 흘림 상놈, 배송도 엄청 빨리 왔고..."
1,순수원목 선반장 4colors (가로2단),29225,"[여기서 날파리가많이나왔어욥왤까여, 이 가격에 이렇게 예뿌다니 진짜 맘에 들어용 품..."
2,오로라 단스탠드(전구 증정 이벤트),20335,[실제로 보니까 더 이쁘네용 ㅋㅋ특히 밤에 조명킬때가 넘 예뻐용💓배송도 엄청빨라서 ...
3,애슐리 전신거울 3colors,14545,[ 🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬🤬 이거 사는 사람 많은거 같은데 불량 나는 안오겠...
4,"집순이 슬리브 옥스포드 삼각 등쿠션 (솜포함, 2size / 6color)",14491,[등받이 길죽한베개로사용하다가 너무불편해서 길죽이베개버리고 고민끝에 주문햇어요 리뷰...


### 각 상품별 keyword 추출하기

In [209]:
print(*df_rev.review.iloc[0])

샤랄라한 분위기 연출해주고 예뻐요 저는 치수를 더 큰것을 사야했는데 잘못맞춰서 아쉽지만요ㅠ 가격대비 이주 잘 산것 같아요 십년만에 남이 준 커튼에서 새로 샀네요 하얀색이 너무 이쁘고 저희 거실 창이 아주 넓은데도 충분히 커버가 되네요 딸들이 너무 이쁘다고 앞에서 틱톡찍어요 가격 생각하면 너무 싸게 잘 샀다고 여겨지네요 넘무 예뻐요 진짜 가성비 짱인 것 같아용 사진이 어두워서그런데 색이쁘고 분홍색ㄷ샀는데 그것도이뻣어요근데 여름이라그런지 좀두꺼워서 아직은 커텐을 쳐놓고 살아욘 ㅎㅎㅎㅎ 완전대만족 남편몰래 서프라이즈해주려고 혼자서 했는데 남편이 넘 좋아해요 어디서 이가격에 커튼을 구입했냐며 너무예쁘다고 하네요 안방이랑 작은방은 작은사이즈로 또 주문하려구여 진짜 백만배강추🥰🥰🥰😍😍😍😍😍 예쁘네요 그런데 대가 좀 약한거 같아요 너무너무너무만족해요진짜 생각한것보다 다 너무예뻐요 ㅠㅠㅠ 이제 안방이랑 작은방 다용도실것도 주문하려구요 완전대만족 추천합니다 이 가격에 이 커튼 못사요 커튼 진짜 특가로 잘샀어요고급지고 집도 넓어보이고 너무 이뻐요 기존에 있던 커튼은 암막이 하나도 안되어서 사봤는데 너무 예쁘고 웨딩드레스처럼 샤랄라한게 대만족이에요 아쉬운건 세로길이가 치수로 재면 딱 맞아야 하는데 아일렛 부분이 높아서 그런지 딸랑하네요 ㅠㅠ 그래도 길어서 머리카락 엉켜붙는것보다는 낫겠거니 하고 그대로 달았어요 진작 살껄 ㅠㅠㅠ 넘 예뻐요 ㅠㅠㅠ 샤랄라 이중레이스 안방 침실용으로 달았습니다 이쁘네요 화사하고 기존 암막커튼은 한 낮에도 쳐놓으면 밤인지 낮인지 몰라서 낮잠자기 좋은 요요 커튼은 암막의 효과보단 인테리어용 으로 좋아요 밝고 화사한 샤랄라느낌으로 분위기 바꿔봤어요 침구랑 잘 어울립니다 이사오면서 주문한건데 사이즈 완전 찰떡 넘나 이뻐요 배송이 빠르긴했는데 상자는 여기저기 뜯어져서 왔더라구요 그래도 빠진것없이 다 잘왔어요 커튼은 예쁘구 설치도 어렵지 않았는데 봉이 엄청 내구성 있어보이진않네요 스팀다림질 한번 해주면 더 예쁠것같아요 사진은 약간 베이지빛나는 흰색인줄 알았는데 

In [233]:
# 0번상품 리뷰에 대한 morphs, nouns, phrases
okt.morphs(df_rev.review.iloc[0][1]) # 해당 n번째 제품의 리뷰 개수만큼 돌려야하는듯...그럼 굳이 review들을 reviews로 합친의의가...ㅠ

['십년',
 '만에',
 '남',
 '이',
 '준',
 '커튼',
 '에서',
 '새로',
 '샀네요',
 '하얀색',
 '이',
 '너무',
 '이쁘고',
 '저희',
 '거실',
 '창',
 '이',
 '아주',
 '넓은데도',
 '충분히',
 '커버',
 '가',
 '되네요',
 '딸',
 '들',
 '이',
 '너무',
 '이쁘다고',
 '앞',
 '에서',
 '틱톡',
 '찍어요',
 '가격',
 '생각',
 '하면',
 '너무',
 '싸게',
 '잘',
 '샀다고',
 '여겨지네요']

In [235]:
r01 = ""
for r in df_rev.review.iloc[0]:
    r01 += r+' '
r01

'샤랄라한 분위기 연출해주고 예뻐요 저는 치수를 더 큰것을 사야했는데 잘못맞춰서 아쉽지만요ㅠ 가격대비 이주 잘 산것 같아요 십년만에 남이 준 커튼에서 새로 샀네요 하얀색이 너무 이쁘고 저희 거실 창이 아주 넓은데도 충분히 커버가 되네요 딸들이 너무 이쁘다고 앞에서 틱톡찍어요 가격 생각하면 너무 싸게 잘 샀다고 여겨지네요 넘무 예뻐요 진짜 가성비 짱인 것 같아용 사진이 어두워서그런데 색이쁘고 분홍색ㄷ샀는데 그것도이뻣어요근데 여름이라그런지 좀두꺼워서 아직은 커텐을 쳐놓고 살아욘 ㅎㅎㅎㅎ 완전대만족 남편몰래 서프라이즈해주려고 혼자서 했는데 남편이 넘 좋아해요 어디서 이가격에 커튼을 구입했냐며 너무예쁘다고 하네요 안방이랑 작은방은 작은사이즈로 또 주문하려구여 진짜 백만배강추🥰🥰🥰😍😍😍😍😍 예쁘네요 그런데 대가 좀 약한거 같아요 너무너무너무만족해요진짜 생각한것보다 다 너무예뻐요 ㅠㅠㅠ 이제 안방이랑 작은방 다용도실것도 주문하려구요 완전대만족 추천합니다 이 가격에 이 커튼 못사요 커튼 진짜 특가로 잘샀어요고급지고 집도 넓어보이고 너무 이뻐요 기존에 있던 커튼은 암막이 하나도 안되어서 사봤는데 너무 예쁘고 웨딩드레스처럼 샤랄라한게 대만족이에요 아쉬운건 세로길이가 치수로 재면 딱 맞아야 하는데 아일렛 부분이 높아서 그런지 딸랑하네요 ㅠㅠ 그래도 길어서 머리카락 엉켜붙는것보다는 낫겠거니 하고 그대로 달았어요 진작 살껄 ㅠㅠㅠ 넘 예뻐요 ㅠㅠㅠ 샤랄라 이중레이스 안방 침실용으로 달았습니다 이쁘네요 화사하고 기존 암막커튼은 한 낮에도 쳐놓으면 밤인지 낮인지 몰라서 낮잠자기 좋은 요요 커튼은 암막의 효과보단 인테리어용 으로 좋아요 밝고 화사한 샤랄라느낌으로 분위기 바꿔봤어요 침구랑 잘 어울립니다 이사오면서 주문한건데 사이즈 완전 찰떡 넘나 이뻐요 배송이 빠르긴했는데 상자는 여기저기 뜯어져서 왔더라구요 그래도 빠진것없이 다 잘왔어요 커튼은 예쁘구 설치도 어렵지 않았는데 봉이 엄청 내구성 있어보이진않네요 스팀다림질 한번 해주면 더 예쁠것같아요 사진은 약간 베이지빛나는 흰색인줄 알았는데

In [236]:
print(okt.morphs(r01))
print()
print(okt.nouns(r01))
print()
print(okt.phrases(r01))

['샤랄라', '한', '분위기', '연출', '해주고', '예뻐요', '저', '는', '치수', '를', '더', '큰것을', '사야', '했는데', '잘', '못', '맞춰서', '아쉽지만', '요', 'ㅠ', '가격', '대비', '이주', '잘', '산것', '같아요', '십년', '만에', '남', '이', '준', '커튼', '에서', '새로', '샀네요', '하얀색', '이', '너무', '이쁘고', '저희', '거실', '창', '이', '아주', '넓은데도', '충분히', '커버', '가', '되네요', '딸', '들', '이', '너무', '이쁘다고', '앞', '에서', '틱톡', '찍어요', '가격', '생각', '하면', '너무', '싸게', '잘', '샀다고', '여겨지네요', '넘', '무', '예뻐요', '진짜', '가성', '비', '짱', '인', '것', '같아용', '사진', '이', '어두워서', '그런데', '색', '이쁘고', '분홍색', 'ㄷ', '샀는데', '그것', '도', '이', '뻣', '어', '요', '근데', '여름', '이라', '그런지', '좀', '두꺼워서', '아직', '은', '커텐', '을', '쳐놓고', '살아욘', 'ㅎㅎㅎㅎ', '완전', '대', '만족', '남편', '몰래', '서프라이즈', '해주려고', '혼자', '서', '했는데', '남편', '이', '넘', '좋아해요', '어디서', '이', '가격', '에', '커튼', '을', '구입', '했냐', '며', '너무', '예쁘다고', '하네요', '안방', '이랑', '작은방', '은', '작은', '사이즈', '로', '또', '주문', '하려구여', '진짜', '백만배', '강추', '🥰🥰🥰😍😍😍😍😍', '예쁘네요', '그런데', '대가', '좀', '약한거', '같아요', '너무', '너무', '너무', '만족해요', '진짜', '생각', '한', '것', '보다', '다', '너무', '예뻐요', 'ㅠㅠㅠ', '이제'

In [225]:
type(ko_stopwords)

list

In [37]:
# vec = CountVectorizer(stop_words=ko_stopwords).fit(df_rev.review.iloc[0])
vec = CountVectorizer(ngram_range=(2,2), stop_words=ko_stopwords).fit(df_rev.review.iloc[0])
bag_of_words = vec.transform(df_rev.review.iloc[0])
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x:x[1], reverse=True)
words_freq[:20]

C:\Users\newel\anaconda3\envs\han\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '대해', '되어', '들면', '들자면', '듯하다', '따르는', '따지지', '때가', '만은', '만이', '만큼', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '바꾸어서', '바꿔', '밖에', '방면으로', '보아', '비길수', '비추어', '뿐만', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않도록', '알겠는가', '어찌됏든', '없고', '없다', '예를', '외에', '위에서', '이와', '이유는', '임에', '있다', '점에서', '주저하지', '줄은', '틀림없다', '편이', '하기', '하는', '하는것만', '하는것이', '하지', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[('너무 좋아요', 1000),
 ('배송도 빠르고', 845),
 ('맘에 들어요', 634),
 ('마음에 들어요', 619),
 ('저렴한 가격에', 354),
 ('너무 마음에', 262),
 ('부드럽고 좋아요', 260),
 ('마음에 듭니다', 256),
 ('크기도 적당하고', 231),
 ('가격도 저렴하고', 227),
 ('너무 맘에', 217),
 ('맘에 듭니다', 213),
 ('너무 좋네요', 210),
 ('가격대비 너무', 192),
 ('촉감도 좋고', 187),
 ('좋아요 ㅎㅎ', 183),
 ('너무 예뻐요', 171),
 ('가성비 좋아요', 158),
 ('가격대비 좋아요', 146),
 ('너무 이뻐요', 141)]

In [38]:
for i, v in enumerate(words_freq):
    if i == 20:
        break
    print(v[0], end=', ')
print()

너무 좋아요, 배송도 빠르고, 맘에 들어요, 마음에 들어요, 저렴한 가격에, 너무 마음에, 부드럽고 좋아요, 마음에 듭니다, 크기도 적당하고, 가격도 저렴하고, 너무 맘에, 맘에 듭니다, 너무 좋네요, 가격대비 너무, 촉감도 좋고, 좋아요 ㅎㅎ, 너무 예뻐요, 가성비 좋아요, 가격대비 좋아요, 너무 이뻐요, 


In [286]:
vec = CountVectorizer(ngram_range=(1,1), stop_words=ko_stopwords).fit(df_rev.review.iloc[3019])
bag_of_words = vec.transform(df_rev.review.iloc[3019])
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x:x[1], reverse=True)
words_freq[:20]

[('이뻐요', 2),
 ('분위기', 1),
 ('있네요', 1),
 ('라탄', 1),
 ('흔들의자랑', 1),
 ('잘어울려요', 1),
 ('러그', 1),
 ('덕분에', 1),
 ('방이', 1),
 ('완벽해졌습니다', 1),
 ('너무', 1),
 ('예뻐요', 1),
 ('부드럽고', 1),
 ('색상', 1),
 ('화면보다', 1),
 ('푸른빛인데', 1),
 ('빈티지하고', 1),
 ('로맨틱하네요', 1),
 ('진짜', 1),
 ('촉감', 1)]

In [40]:
# df_rev shape:
df_rev.shape

(3650, 3)

### 3650개 상품에 대한 각각 top 10 review bigram

In [234]:
# most_review drop하기
df_rev = df_rev.drop(columns='most_review')

In [235]:
df_rev.head(1)

,상품명,review
0,선데이 러그 7size 4colors,"[너무 좋네요 선배가 와서 양념치킨 먹다가 바로 흘림 상놈, 배송도 엄청 빨리 왔고..."


In [236]:
df_rev.insert(2, "most_review", "nothing")
df_rev.head(2)

,상품명,review,most_review
0,선데이 러그 7size 4colors,"[너무 좋네요 선배가 와서 양념치킨 먹다가 바로 흘림 상놈, 배송도 엄청 빨리 왔고...",nothing
1,순수원목 선반장 4colors (가로2단),"[여기서 날파리가많이나왔어욥왤까여, 이 가격에 이렇게 예뿌다니 진짜 맘에 들어용 품...",nothing


In [115]:
for i in range(df_rev.shape[0]):
    try:
        vec = CountVectorizer(ngram_range=(2,2), stop_words=ko_stopwords).fit(df_rev.review.iloc[i])
        bag_of_words = vec.transform(df_rev.review.iloc[i])
        sum_words = bag_of_words.sum(axis=0)
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq = sorted(words_freq, key = lambda x:x[1], reverse=True)[:10]
        df_rev.at[i, 'most_review'] = [row[0] for row in words_freq][:5] # ValueError: empty vocabulary; perhaps the documents only contain stop words
        if len(df.iloc[i]['review']) < 11:
            # 리뷰개수가 적어 겹칠경우 방지를 위해 홀수번째만
            df_rev.at[i, 'most_review'] = [row[0] for row in words_freq][::2]
    except:
        df_rev.at[i, 'most_review'] = [""] # ValueError: empty vocabulary; perhaps the documents only contain stop words

C:\Users\newel\anaconda3\envs\han\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '대해', '되어', '들면', '들자면', '듯하다', '따르는', '따지지', '때가', '만은', '만이', '만큼', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '바꾸어서', '바꿔', '밖에', '방면으로', '보아', '비길수', '비추어', '뿐만', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않도록', '알겠는가', '어찌됏든', '없고', '없다', '예를', '외에', '위에서', '이와', '이유는', '임에', '있다', '점에서', '주저하지', '줄은', '틀림없다', '편이', '하기', '하는', '하는것만', '하는것이', '하지', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [118]:
df_most = df_rev[['pname', 'most_review']]
df_most.to_excel("각 상품별 상위5개 bigram 키워드.xlsx")
df_most.sample(10)

,pname,most_review
850,KURBY 의자 A타입 3colors,"[배송도 빠르고, 마음에 들어요, 너무 예뻐요, 맘에 들어요, 맘에 듭니다]"
1435,하멜 리본타이 린넨쿠션(솜포함)(50x50),"[너무 예뻐요, 배송도 빠르고, 마음에 들어요, 너무 맘에, 너무 빵빵해서]"
248,PP 라탄 바스켓 수납바스켓,"[너무 좋아요, 뚜껑이 있어서, 너무 예뻐요, 마음에 들어요, 맘에 들어요]"
2442,크리스마스트리 샬롯레드 1.3M 전구장식 풀세트,"[너무 예뻐요, 생각했던거 보다, 보다 퀄리티가, 퀄리티가 너무, 너무 좋아서]"
1880,원목 라탄 나트랑체어 2colors,"[너무 편하고, 너무 예뻐요, 마음에 듭니다, 처음에 불량제품이, 불량제품이 와서]"
1586,HUGGY BEAR FABRIC POSTER_2sizes,"[너무 귀여워요, 너무 귀엽고, 가격은 비싸지만, 너무 귀여워서, 맘에 들어요]"
1001,[할인쿠폰] 버니 방수 패브릭 3.5인 카우치 소파,"[맘에 들어요, 기다린 보람이, 너무 이뻐요, 맘에 듭니다, 너무 맘에]"
3033,[쿠폰할인] 뉴플로렌스 골드 스탠딩 거울,"[생각햇던 금색은, 금색은 아니지만, 아니지만 무광황색, 무광황색 이게, 이게 이쁜것]"
3601,[해외] Setago JH27 테이블 램프 nude-forest,"[디자인 마음에, 마음에 듭니다, 듭니다 무선이라서, 무선이라서 휴대성이, 휴대성이..."
506,노르딕 육각 테이블 5colors,"[조립도 쉽고, 너무 좋아요, 맘에 들어요, 배송도 빠르고, 너무 예뻐요]"


In [104]:
df_rev.iloc[2222]['review']

['생각보다 작지만 군더더기 없이 귀엽네요 잘 쓸게요',
 '보온도 잘 되고 디자인도 아담하니 예쁩니다',
 '사이즈 딱 좋고 죽통으로 괜찮을 것 같아요  색감도 파스텔 톤이라 예쁘고 배송도 하루만에 도착 했어요 ',
 '예쁘고 사용하기 편하네요 밀폐력도 좋은거 같아요',
 '텀블러만 주는줄 알았는데 사은품으로 커버랑 수저도 포함되어 있었네요이럴줄알았으면 커버는 따로 사지 말껄 그랬니봐요ㅠㅠ다행히 색상은 다르게와서 기분전환할때마다 다르게 껴야겠어요 ㅋㅋ 쨌든 사은품 감사드리고 잘쓰겠습니다',
 '좋아요좋아요좋아요좋아요좋아요좋아요',
 '모슈 텀블러 사용 했는데 보온 보냉이 잘 돼서 죽통도 주문 했어요 아직 사용을 안 해봐서 성능은 모르겠지만 디자인은 너무 귀여워요',
 '예뻐서 샀어요 보온ㅇ력은 평범한거같아요',
 '귀엽고 이뻐요 색감도 이쁘네요 맘에듭니다',
 '너무 귀엽네요 생각보다 많이 들어갈것같아서 더욱 좋아요',
 '디자인도 귀엽고 같이 동봉되어 온 숟가락도 너무 귀엽습니다 귀여운걸 떠나서 보온이 정말 잘되네요 아침에 만든 수프를 넣어서 점심에 먹는데도 뜨겁습니다 만족해요',
 '디자인 너무 심플하고 색상도 이뻐요',
 '제가 생각한 민트하고는 완전다르네요디자인은 예뻐요보온력은 그냥 그래요',
 '배송이 빨라서 좋았구요 크기도 딱 원하는 사이즈예요 스푼이랑 커버도 너무 귀엽네요 모슈제품 많이 가지고 있는데 디자인이 참 이뻐요ㅋ',
 '모양도 이쁘고 양이 적절해요 비비고 미역국 한 봉이 그대로 다 들어갑니다 제법 온기도 오래가서 도시락용으로 좋아요',
 '빠른배송 감사합니다 이뻐요 420L이 혹시 클까봐 걱정했는데 큰거하기를 잘한것같아요',
 '색상 너무 마음에 들어요 교정하고 먹는게 불편해서 죽 먹는데 딱 좋아요',
 '요즘 도시락 싸서 출근하는데 보온도시락이 필요하겠더라그용 다른 거랑 고민하다가 이게 이뻐서 걍 주문했어요 ㅜㅜ 이뻐요 잘 쓸게요',
 '아기이유식용으로산건데 넘나이쁘고 보온잘되요',
 '넘넘 예뻐요 깨물어주고 싶엉 머스터드 컬러는 뚜껑

In [109]:
df_rev.most_review

0       [너무 좋아요, 배송도 빠르고, 맘에 들어요, 마음에 들어요, 저렴한 가격에, 너무...
1       [조립도 쉽고, 배송도 빠르고, 마음에 들어요, 너무 좋아요, 맘에 들어요, 침대 ...
2       [너무 예뻐요, 배송도 빠르고, 맘에 들어요, 너무 이뻐요, 너무 좋아요, 마음에 ...
3       [조립도 쉽고, 너무 예뻐요, 배송도 빠르고, 맘에 들어요, 마음에 들어요, 너무 ...
4       [너무 좋아요, 편하고 좋아요, 너무 편하고, 너무 편해요, 배송도 빠르고, 생각보...
                              ...                        
3645                    [토퍼가좀딱따해요 부드러운토퍼, 부드러운토퍼 다시사야겧어요]
3646                                [깔끔하고 디자인, 좋네요 만족합니다]
3647             [가격대비 만족해요, 만족해요 배송도괜찮고, 배송도괜찮고 사용하기좋네요]
3648            [배송도 빨리오고, 빨리오고 가격도, 가격도 저렴해서, 저렴해서 좋습니다]
3649    [일단 디자인, 디자인 적으로, 적으로 심플해서, 심플해서 마음에, 마음에 들었고요...
Name: most_review, Length: 3650, dtype: object

In [227]:
df_rev.sample(20)

,상품명,review,most_review
2831,[리퍼] 공기청정기 Le-510 원룸 H13헤파필터,[행사로 저렴하게 잘 샀습니다 자취하면서 작은 청정기 알아보고 있었는데 uv기능보고...,nothing
2557,라탄등나무피크닉바구니,"[예뻐요 저는 참숯을 담아놨는뎅 너무 예뻐요, 원하던색상에 디자인까지, 예뻐요 잘 ...",nothing
1463,원목 협탁 사이드 테이블 논슬립 스펀지 증정 (2type),[선물용으러 구입한건대 볼트랑 꺽쇠랑 사이즈도 안맞고 이건 조립을 어떻게 하라는건지...,nothing
3097,가드너스와이프 유약분 -- 화이트,"[작은 사이즈는 기본 화원포트 분갈이용으로 적합해요 너무 이뻐요❣️, 깔끔하고 흠잡...",nothing
1204,그리너리 가리개 커튼 (창문형/긴창형),[생각보다 훨씬 장화홍련느낌이라 첨에 약당황했으나 소재가 너무 좋고 고급져보여요 책...,nothing
1956,allday LIM 브런치 플레이트,"[너무 귀여워요 밀크티베이지인가 이색은 귀여워서 있는 것 거의 다 샀네요, 기대이상...",nothing
2250,보르미올리 비스트로 저그 유리물병 1.0L,[작은것 하나1L 두개 구매했어요 너무 이쁘게 잘 쓰고있습니다 유리 단단하고 깨지지...,nothing
2689,[해외] 전기포트 KLF03 8color,"[배송도 완전 빠르고 좋네요 강추합니다, 꺄 넘이뽀요 착각해서 도착도 하기전에 구매...",nothing
2592,행잉 레더스트랩 (L) - HANGING LEATHER STRAP (L),[아주 아주 아주 맘에 들어요 행잉 레더스트랩 이것저것 많이 보고 비교해봤는데 정말...,nothing
74,[쿠폰할인] 피카 사이드 테이블,[조립 직접하시는분들 꼭 보셨으면 여자혼자사는집이라 기사님께 비밀번호 알려드리긴 좀...,nothing


In [ ]:
# vec = CountVectorizer(stop_words=ko_stopwords).fit(df_rev.review.iloc[1])
vec = CountVectorizer(ngram_range=(2,2), stop_words=ko_stopwords).fit(df_rev.review.iloc[1])
bag_of_words = vec.transform(df_rev.review.iloc[0])
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items() if ]
words_freq = sorted(words_freq, key = lambda x:x[1], reverse=True)
words_freq[:20]

# KRWordRank 시도
- [https://github.com/lovit/KR-WordRank](https://github.com/lovit/KR-WordRank) 코드 그대로 사용

In [121]:
from krwordrank.word import KRWordRank

In [172]:
min_count = 5
max_length = 300
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

In [198]:
beta = 0.85
max_iter = 100
texts = df_rev["review"][3019]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

In [200]:
# 불용어 정의(리뷰에서 너무 많이 보여거 뺄 것)
custom_stopwords = {'너무', 'ㅎㅎ', 'ㅋㅋ', '좋아요', '예뻐요', '이뻐요', '진짜', '정말', '예쁘고', 
                    '이쁘고', '맘에','들어요', '완전', '좋고', '생각보다', '같아요', '마음에',
                    '조금', '엄청', '아주', '좋습니다', '좋네요', '생각', '예쁘'}
passwords = {word:score for word, score in 
             sorted(keywords.items(), key=lambda x:-x[1])[:20] if not (word in custom_stopwords)}
passwords

{}

In [186]:
df_rev.shape

(3650, 4)

In [288]:
df_rev[df_rev['상품명'] == "빈티지그린 로즈부케 드로잉 원형러그"]['review']

3019    [분위기 있네요 라탄 흔들의자랑 잘어울려요, 러그 덕분에 제 방이 완벽해졌습니다 너...
Name: review, dtype: object

In [289]:
df_rev["review"][3019]

['분위기 있네요 라탄 흔들의자랑 잘어울려요',
 '러그 덕분에 제 방이 완벽해졌습니다 너무 예뻐요 ',
 '부드럽고 색상 넘 이뻐요 화면보다 좀 더 푸른빛인데 빈티지하고 로맨틱하네요',
 '진짜 촉감 너어어어무 좋아요  거실카펫도 여기꺼로 바꾸려구요 거기다가 세탁기돌려도 된다니 대만족입니다 ㅋㅋ 실물이 훨 이뻐요']

In [201]:
# without stopwords
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      너무:	2.3830


In [287]:
# summarize_with_keywords 함수
from krwordrank.word import summarize_with_keywords

keywords = summarize_with_keywords(df_rev["review"][3019], min_count=3, max_length=10,
           beta=0.85, max_iter=10, stopwords=custom_stopwords, verbose=True)

scan vocabs ... 
num vocabs = 3
done = 1 Early stopped.


In [283]:
# print([kwrd for kwrd in keywords][:5])

# for i in range(len(keywords)):
#     print(keywords[i])

print(df_rev["상품명"][77][7:])
keywords

크리미 세미워싱 차렵이불 (S/Q) 2colors


{'이불': 14.644398451993231,
 '가볍고': 12.997023166649319,
 '부드럽고': 7.685038654354408,
 '느낌': 6.251748765404665,
 '색도': 5.484756456718726,
 '포근': 5.17273446982149,
 '배송': 4.654156369794625,
 '약간': 4.638713407702633,
 '세탁': 4.3998864927492605,
 '만족': 4.383897975177778,
 '촉감': 4.069411973984497,
 '베이지': 3.9640799566493943,
 '색상도': 3.854979853034597,
 '그래도': 3.753163479791768,
 '폭신': 3.7266687842778823,
 '가격': 3.706242314847995,
 '따뜻': 3.537542457177348,
 'ㅠㅠ': 3.34983129129913,
 '구매': 3.3497817387426396,
 '근데': 3.3460161689032053,
 '제가': 3.275347530812946,
 '많이': 3.2628453350366025,
 '색깔도': 3.1369929913288233,
 '가성비': 3.0247877446579294,
 '부들': 2.95640023620295,
 '그냥': 2.944697751342009,
 '보들': 2.8929053051123153,
 '사용': 2.7824838512566137,
 '바로': 2.7701940486195022,
 '아직': 2.758731219274284,
 '좋은': 2.7231358936021763,
 '색감': 2.674966054899416,
 '푹신': 2.5309616139586923,
 '다른': 2.47217981811262,
 '색은': 2.468087812369636,
 '잠이': 2.416919581603841,
 '빠르고': 2.3887785622676203,
 '패드': 2.24463774

# 찐 키워드 추출
- KRWordRank
- 아래 두 셀이 다임

In [255]:
stopwords = {'너무', 'ㅎㅎ', 'ㅎㅎㅎ', 'ㅋㅋ', '좋아요', '예뻐요', '이뻐요', '진짜', '정말', '예쁘고', 
                    '이쁘고', '맘에','들어요', '완전', '좋고', '생각보다', '같아요', '마음에', 'ㅜㅜ', 'ㅜㅜㅜ'
                    '조금', '엄청', '아주', '좋습니다', '좋네요', '생각', '예쁘', 'ㅠㅠ', 'ㅠㅠㅠ', 'ㅋㅋㅋ'}

In [256]:
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords

# 모든 상품 각각에 대한 top5 키워드 추출
for i in range(df_rev.shape[0]):
    keywords = summarize_with_keywords(df_rev["review"][i], 
                                       min_count=1, max_length=10, 
                                       beta=0.85, max_iter=10, 
                                       stopwords=stopwords, verbose=True)
    # 각 상품명 row별 most reviewed 항목 value값 변경
    df_rev.at[i, 'most_review'] = [kwrd for kwrd in keywords][:5] # 상위 5개만

scan vocabs ... 
num vocabs = 315
done = 10
scan vocabs ... 
num vocabs = 429
done = 10
scan vocabs ... 
num vocabs = 589
done = 10
scan vocabs ... 
num vocabs = 1072
done = 10
scan vocabs ... 
num vocabs = 495
done = 10
scan vocabs ... 
num vocabs = 911
done = 10
scan vocabs ... 
num vocabs = 370
done = 10
scan vocabs ... 
num vocabs = 380
done = 10
scan vocabs ... 
num vocabs = 258
done = 10
scan vocabs ... 
num vocabs = 249
done = 10
scan vocabs ... 
num vocabs = 480
done = 10
scan vocabs ... 
num vocabs = 350
done = 10
scan vocabs ... 
num vocabs = 377
done = 10
scan vocabs ... 
num vocabs = 323
done = 10
scan vocabs ... 
num vocabs = 500
done = 10
scan vocabs ... 
num vocabs = 330
done = 10
scan vocabs ... 
num vocabs = 281
done = 10
scan vocabs ... 
num vocabs = 608
done = 10
scan vocabs ... 
num vocabs = 469
done = 10
scan vocabs ... 
num vocabs = 411
done = 10
scan vocabs ... 
num vocabs = 308
done = 10
scan vocabs ... 
num vocabs = 410
done = 10
scan vocabs ... 
num vocabs = 2

In [ ]:
kr_rev[kr_rev[]'상품명']

In [261]:
df_rev.at[3642, 'most_review'] = ["좋아요"]

In [264]:
# column 두개만 뽑아보기
df_22 = df_rev[['상품명', 'most_review']]
df_22.to_excel("keyword_per_product.xlsx")

In [267]:
print(df_22.iloc[407]['most_review'])

['배송', '가성비', '소파', '만족', '스툴']


In [214]:
df_prod = pd.read_excel("product_data_full.xlsx")
print(df_prod.shape)
df_prod.head(3)

(3596, 21)


,product_ID,상품명,product_price,Product_Url,대분류,대분류_1,중분류,중분류_1,중분류_2,소분류,...,소분류_2,brand,Thema_G,리뷰수,총점,별점평균,내구성,가격,디자인,배송
0,Product_0,디퓨저 리필액 200ml 1+1+1+1 + 캔들증정,14900,https://ohou.se/productions/423747/selling,홈데코,조명,디퓨져,디퓨져,NaN,디퓨져,...,NaN,코코도르,['Romantic'],496,4.7,4.686895,4.725806,4.649194,4.661290,4.677419
1,Product_1,줄무늬 글라스 유리병 2size,2890,https://ohou.se/productions/81677/selling,홈데코,조명,플라워,식물,NaN,화병,...,NaN,코제트,"['Romantic', 'Antique']",982,4.7,4.868432,4.831976,4.891039,4.874745,4.861507
2,Product_2,회전목마캔들 메리고라운드 오브제캔들 디자인 향초 필라왁스 인테리어소품,15800,https://ohou.se/productions/571494/selling,홈데코,조명,캔들,디퓨져,NaN,캔들,...,NaN,F5NATURE,['Romantic'],4,5.0,5.000000,5.000000,5.000000,5.000000,5.000000


In [219]:
df_rev.columns = ['상품명', 'count', 'review']
df_rev = df_rev[['상품명', 'review']]
df_rev.head(2)

,상품명,review
0,선데이 러그 7size 4colors,"[너무 좋네요 선배가 와서 양념치킨 먹다가 바로 흘림 상놈, 배송도 엄청 빨리 왔고..."
1,순수원목 선반장 4colors (가로2단),"[여기서 날파리가많이나왔어욥왤까여, 이 가격에 이렇게 예뿌다니 진짜 맘에 들어용 품..."


### 기존 35nn개 상품 dataframe과 top5 리뷰데이터 dataframe 합치기

In [221]:
df_prod = df_prod.merge(df_rev, on='상품명')
print(df_prod.shape)
df_prod.sample(5)

(3596, 22)


,product_ID,상품명,product_price,Product_Url,대분류,대분류_1,중분류,중분류_1,중분류_2,소분류,...,brand,Thema_G,리뷰수,총점,별점평균,내구성,가격,디자인,배송,review
2204,Product_2204,퍼블릭 원목 접시꽂이 6칸,5700,https://ohou.se/productions/381625/selling,주방,NaN,주방수납,정리,NaN,주방선반,...,키친몰링,['Modern'],34,4.5,4.497059,4.382353,4.382353,4.529412,4.647059,"[원하는 디자인이구요 타 사이트보다 저렴하게 구입했어요, 도마받침대로 쓰고있어요 튼..."
393,Product_393,트리 오너먼트 19종,2900,https://ohou.se/productions/175443/selling,홈데코,조명,크리스마스,NaN,NaN,트리,...,아웃팅,['Romantic'],3,3.3,2.600000,3.000000,2.333333,3.000000,2.000000,[일단 구성에서 솔방울 빠져서 안왔구요 저는 원래 있는 구성에다가 약 삼천 원어치 ...
82,Product_82,"원프레드화이트 쇼파커버 (거실러그, 블랭킷, 다용도 패브릭)",34900,https://ohou.se/productions/106089/selling,패브릭,NaN,홈패브릭,NaN,NaN,소파커버,...,레인보우하우스,"['Romantic', 'Natural', 'Modern']",1043,4.6,4.574497,4.256951,4.549377,4.725791,4.710451,"[쇼파가 금방 지저분해질거 같아서 깔았는데 너무 잘어울리고 사이즈도 딱이네요, 쇼파..."
2298,Product_2298,뮤즈 고밀도 바이오워싱 순면 호텔식 차렵이불 세트 S/Q,117000,https://ohou.se/productions/346431/selling,패브릭,NaN,침구,NaN,NaN,차렵이불,...,에이트룸,"['Antique', 'Modern']",492,4.6,4.602236,4.485772,4.532520,4.752033,4.599593,[다른 이불들하고 엄청 비교해보다가 구매했는데 받아보니까 너무 좋아요 겉은 바스락거...
587,Product_587,[쿠폰할인][단독] 아만다 시크릿수납 원목 화장대 1000(스툴포함),149000,https://ohou.se/productions/315517/selling,가구,NaN,침실가구,NaN,NaN,화장대,...,잉글랜더,['Romantic'],199,4.5,4.421608,4.537688,4.457286,4.753769,3.879397,[고급스러운 원목 화장대가 집안 분위기를 따뜻하게해줘요ㅎㅎ 수납공간도 많고 만족합니...
